In [5]:
import requests


PACE = 'pace'
SHOT = 'shot'
PASS = 'pass'
DRIBBLE = 'dribble'
DEFENSE = 'defense'
PHYSICAL = 'physical'

data = []
for i in range(1, 5):
    response = requests.get('https://www.easports.com/fifa/ultimate-team/api/fut/item?page={}'.format(i))
    json = response.json()
    data.append(json)

items = [item['items'] for item in data]
items = [item for sublist in items for item in sublist]

In [3]:
keys = ['attributes', 'rating']
players = []
for item in items:
    player = dict()
    if not item['commonName']:
        player.update({'name': item['firstName'] + ' ' + item['lastName']})
    else:
        player.update({'name': item['commonName']})
    for key in keys:
        player.update({key: item[key]})
    for attribute in item['attributes']:
        attribute_name = attribute['name']
        attribute_value = attribute['value']
        attributes = dict()
        attributes['value'] = attribute_value
        if attribute_name == 'fut.attribute.pac'
    players.append(player)
            
# print(players)
    

fut.attribute.PAC
fut.attribute.SHO
fut.attribute.PAS
fut.attribute.DRI
fut.attribute.DEF
fut.attribute.PHY
fut.attribute.PAC
fut.attribute.SHO
fut.attribute.PAS
fut.attribute.DRI
fut.attribute.DEF
fut.attribute.PHY
fut.attribute.PAC
fut.attribute.SHO
fut.attribute.PAS
fut.attribute.DRI
fut.attribute.DEF
fut.attribute.PHY
fut.attribute.PAC
fut.attribute.SHO
fut.attribute.PAS
fut.attribute.DRI
fut.attribute.DEF
fut.attribute.PHY
fut.attribute.PAC
fut.attribute.SHO
fut.attribute.PAS
fut.attribute.DRI
fut.attribute.DEF
fut.attribute.PHY
fut.attribute.PAC
fut.attribute.SHO
fut.attribute.PAS
fut.attribute.DRI
fut.attribute.DEF
fut.attribute.PHY
fut.attribute.PAC
fut.attribute.SHO
fut.attribute.PAS
fut.attribute.DRI
fut.attribute.DEF
fut.attribute.PHY
fut.attribute.PAC
fut.attribute.SHO
fut.attribute.PAS
fut.attribute.DRI
fut.attribute.DEF
fut.attribute.PHY
fut.attribute.PAC
fut.attribute.SHO
fut.attribute.PAS
fut.attribute.DRI
fut.attribute.DEF
fut.attribute.PHY
fut.attribute.PAC
fut.attrib